In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import pyaudio
import wave
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/home/falcon/Documents/Office Docs/POCs/Speech Emotion Recognition/Datasets/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        print(feature.shape)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.2)

(40,)
(0,)
[-6.24773193e+02  6.18993988e+01  1.61996670e+01  2.69222374e+01
  1.14536390e+01  9.72471142e+00 -3.24117112e+00 -4.94035244e+00
  5.99512815e+00 -2.62095761e+00 -1.85189366e+00  6.24902868e+00
 -4.45410776e+00  1.74355280e+00 -3.19657135e+00  1.54662266e-01
 -2.25432348e+00  3.13730526e+00 -1.53397262e+00  5.82141042e-01
 -1.55462396e+00  7.02494860e-01 -3.36183794e-02 -1.09871423e+00
 -2.20549750e+00 -1.95055962e+00 -3.43158364e+00 -2.83004880e-01
 -1.51648521e+00  1.62102020e+00 -8.12076628e-01 -3.14289069e+00
 -2.39280438e+00  2.36586913e-01 -3.11449337e+00 -1.85444057e+00
 -1.18769348e+00 -1.12029290e+00 -2.89225554e+00 -1.99549556e+00]
[-6.24773193e+02  6.18993988e+01  1.61996670e+01  2.69222374e+01
  1.14536390e+01  9.72471142e+00 -3.24117112e+00 -4.94035244e+00
  5.99512815e+00 -2.62095761e+00 -1.85189366e+00  6.24902868e+00
 -4.45410776e+00  1.74355280e+00 -3.19657135e+00  1.54662266e-01
 -2.25432348e+00  3.13730526e+00 -1.53397262e+00  5.82141042e-01
 -1.55462396e

/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[6.1035156e-05 3.0517578e-05 6.1035156e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[ 0.00085449  0.00064087 -0.00012207 ... -0.00021362 -0.00024414
 -0.00024414] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[-1.8310547e-04 -1.2207031e-04 -3.0517578e-05 ... -1.5258789e-04
 -2.1362305e-04 -1.8310547e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/tmp/ipykernel_41098/2064313628.py:22

(40,)
(0,)
[-447.53927612   29.3398304    -7.01418257    5.12863445    2.27992034
    0.99858499   -4.03680611   -6.61684752   -1.14026189    3.56760406
   -5.17961216    3.92337418   -3.45287824    1.54985738   -0.745996
   -3.48709512    2.34372473    1.35640657    4.03814983    4.90855455
    5.25648403    4.52242136    5.03891516    6.8985157     6.89613724
    7.29328012    5.00084639    6.14949036    4.34518766    6.56104803
    4.29503489    2.82362032    2.13452911    4.53861284    3.58846593
    3.65914297    2.37441301    5.55359411    4.78858805    3.53002667]
[-447.53927612   29.3398304    -7.01418257    5.12863445    2.27992034
    0.99858499   -4.03680611   -6.61684752   -1.14026189    3.56760406
   -5.17961216    3.92337418   -3.45287824    1.54985738   -0.745996
   -3.48709512    2.34372473    1.35640657    4.03814983    4.90855455
    5.25648403    4.52242136    5.03891516    6.8985157     6.89613724
    7.29328012    5.00084639    6.14949036    4.34518766    6.5610480

/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[-0.00033569 -0.00021362  0.         ...  0.00021362  0.00021362
  0.00021362] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[0.00027466 0.00027466 0.00027466 ... 0.00018311 0.00018311 0.00012207] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[ 9.1552734e-05  6.1035156e-05  9.1552734e-05 ... -3.0517578e-04
 -3.0517578e-04 -3.0517578e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pa

[-5.67380432e+02  5.15668373e+01 -2.09014106e+00  1.85368366e+01
  7.01540899e+00  6.42165947e+00 -5.90052128e+00 -8.10319519e+00
  4.64251709e+00  2.10655618e+00 -5.95970821e+00  5.89060688e+00
 -2.31533527e+00  3.10432005e+00 -1.91658342e+00 -2.95350194e+00
 -2.62321210e+00 -8.63706827e-01 -2.96532822e+00 -1.62392116e+00
 -7.10660458e-01  5.16383171e-01 -8.89910877e-01 -5.97295880e-01
 -1.06560636e+00  8.86924863e-01 -6.67404711e-01  3.64182782e+00
  3.33930683e+00  2.48241067e+00  1.14302719e+00  1.43837237e+00
  1.58978617e+00  2.09475708e+00  4.15751562e-02  2.47123218e+00
  2.83771420e+00  3.45129204e+00  1.36426473e+00  1.45448744e+00
  5.52011192e-01  5.59411526e-01  5.37382543e-01  5.59625745e-01
  6.08228087e-01  6.94732606e-01  6.65617824e-01  6.76082969e-01
  7.60609806e-01  7.45113969e-01  6.66203737e-01  6.31935418e-01
  4.34240821e-04  3.08915769e-04  1.65792543e-03  6.10060524e-03
  2.55531594e-02  2.34428614e-01  6.82048082e-01  3.91525447e-01
  4.12130922e-01  2.52664

...

(40,)
(0,)
[-5.67400696e+02  3.58359795e+01 -1.41421452e+01  7.07050133e+00
 -1.17057047e+01 -7.80806923e+00 -2.21263294e+01 -1.16542568e+01
 -1.46855032e+00 -9.99248028e+00 -4.99039507e+00 -9.63266754e+00
 -4.50165081e+00 -6.32821465e+00 -9.01575947e+00 -4.51179266e+00
 -6.03864956e+00 -4.73118162e+00  6.07929170e-01  2.24878645e+00
 -6.70502111e-02  1.63783073e+00  1.06816494e+00  3.75286293e+00
  2.71748877e+00  3.77871466e+00  5.60188341e+00  5.60247755e+00
  4.49537420e+00  4.83263540e+00  3.32501364e+00  2.63196230e-01
  4.73244250e-01  3.19547391e+00  1.39610505e+00  1.69084108e+00
  4.41477716e-01  2.70793343e+00  1.67811239e+00  1.74831665e+00]
[-5.67400696e+02  3.58359795e+01 -1.41421452e+01  7.07050133e+00
 -1.17057047e+01 -7.80806923e+00 -2.21263294e+01 -1.16542568e+01
 -1.46855032e+00 -9.99248028e+00 -4.99039507e+00 -9.63266754e+00
 -4.50165081e+00 -6.32821465e+00 -9.01575947e+00 -4.51179266e+00
 -6.03864956e+00 -4.73118162e+00  6.07929170e-01  2.24878645e+00
 -6.70502111e

(40,)
(0,)
[-4.61683289e+02  2.97213726e+01 -2.37437630e+01  7.40782976e+00
 -1.13392925e+01 -6.83393908e+00 -3.72477746e+00 -8.22808170e+00
  1.67640984e+00 -2.03673720e+00  6.58008218e-01 -3.98588324e+00
  1.96054423e+00  4.09150124e+00  7.71801829e-01  9.25092125e+00
  7.67769146e+00  9.46342087e+00  1.00225220e+01  5.12354898e+00
  3.92028618e+00  6.47740650e+00  4.36598206e+00  9.16539097e+00
  5.18011570e+00  6.62470770e+00  3.46421266e+00  5.14127159e+00
  2.70072556e+00  3.29076219e+00  3.41592360e+00  3.91241217e+00
  4.27741718e+00  3.82343984e+00  1.92021573e+00  3.67114258e+00
  3.25211883e-01  8.54221940e-01  2.45944396e-01  1.02180505e+00]
[-4.61683289e+02  2.97213726e+01 -2.37437630e+01  7.40782976e+00
 -1.13392925e+01 -6.83393908e+00 -3.72477746e+00 -8.22808170e+00
  1.67640984e+00 -2.03673720e+00  6.58008218e-01 -3.98588324e+00
  1.96054423e+00  4.09150124e+00  7.71801829e-01  9.25092125e+00
  7.67769146e+00  9.46342087e+00  1.00225220e+01  5.12354898e+00
  3.92028618e

/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[ 0.0000000e+00 -3.0517578e-05  0.0000000e+00 ...  3.0517578e-05
  3.0517578e-05  3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00085449 -0.00088501
 -0.00091553] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


(40,)
(0,)
[-4.69931030e+02  3.02363625e+01 -2.09024715e+01  9.42819500e+00
 -8.72711658e+00 -6.68052435e+00 -2.47888780e+00 -8.33877182e+00
  2.15719032e+00 -2.37342072e+00  1.83878827e+00 -4.16126728e+00
 -1.53750801e+00  1.74357307e+00 -4.40781307e+00  8.26307678e+00
  6.96375608e+00  1.26084280e+01  1.32204199e+01  8.99573231e+00
  6.14205170e+00  6.04934597e+00  2.10163379e+00  6.19489431e+00
  3.38442659e+00  7.06796694e+00  2.90112519e+00  4.55077553e+00
  4.10849190e+00  5.34047651e+00  4.26263237e+00  3.61975980e+00
  4.48261547e+00  4.07228708e+00  1.98464680e+00  2.81251311e+00
  1.50331581e+00  1.66891098e+00  3.08829427e-01  6.82924688e-01]
[-4.69931030e+02  3.02363625e+01 -2.09024715e+01  9.42819500e+00
 -8.72711658e+00 -6.68052435e+00 -2.47888780e+00 -8.33877182e+00
  2.15719032e+00 -2.37342072e+00  1.83878827e+00 -4.16126728e+00
 -1.53750801e+00  1.74357307e+00 -4.40781307e+00  8.26307678e+00
  6.96375608e+00  1.26084280e+01  1.32204199e+01  8.99573231e+00
  6.14205170e

/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00021362 -0.00024414
 -0.00018311] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


In [6]:
x_train.shape

(614, 180)

In [7]:
x_test.shape

(154, 180)

In [8]:
x_test.shape

(154, 180)

In [9]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


In [10]:
x_test.shape

(154, 180)

In [11]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [12]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [13]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [14]:
#Predict for the test set
y_pred=model.predict(x_test)

In [15]:
y_pred

array(['happy', 'disgust', 'disgust', 'happy', 'happy', 'calm', 'calm',
       'fearful', 'disgust', 'disgust', 'fearful', 'disgust', 'happy',
       'disgust', 'disgust', 'fearful', 'calm', 'fearful', 'disgust',
       'disgust', 'fearful', 'disgust', 'fearful', 'disgust', 'disgust',
       'calm', 'disgust', 'fearful', 'disgust', 'happy', 'happy',
       'fearful', 'happy', 'disgust', 'disgust', 'calm', 'fearful',
       'calm', 'disgust', 'disgust', 'happy', 'disgust', 'fearful',
       'happy', 'disgust', 'happy', 'disgust', 'calm', 'disgust', 'calm',
       'disgust', 'disgust', 'disgust', 'happy', 'disgust', 'happy',
       'fearful', 'disgust', 'disgust', 'fearful', 'fearful', 'happy',
       'disgust', 'disgust', 'fearful', 'disgust', 'fearful', 'happy',
       'disgust', 'disgust', 'disgust', 'calm', 'happy', 'fearful',
       'fearful', 'happy', 'disgust', 'fearful', 'disgust', 'fearful',
       'happy', 'happy', 'disgust', 'happy', 'happy', 'disgust', 'calm',
       'disgust

In [16]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 62.34%


In [17]:
#Save the Model
pickle.dump(model, open("weights.pkl", 'wb'))

In [18]:
#Load the Model
model = pickle.load(open("weights.pkl", 'rb'))

In [20]:
filename = "weights.pkl"
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("Datasets/output.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

(40,)
(0,)
[-2.86187744e+02  1.59144608e+02  2.53865643e+01  1.48403635e+01
  2.00531025e+01  1.59656124e+01  2.46856958e-02  2.30877662e+00
  2.62519002e+00 -1.10081995e+00 -1.00896149e+01  6.85040808e+00
  4.45368528e+00 -4.11487961e+00  3.77111149e+00  1.63347983e+00
  1.22343516e+00 -2.25666666e+00  3.82052279e+00  1.57108784e+00
 -4.55190754e+00 -2.27640009e+00 -5.84893286e-01 -6.44204021e-01
 -6.98005855e-01 -1.42450297e+00 -4.44694489e-01  6.53092444e-01
 -1.63652432e+00 -7.98121616e-02  2.28842854e+00  1.57999027e+00
  1.44979596e+00 -2.19269872e-01  2.15632701e+00 -1.39819944e+00
 -7.59551883e-01  1.46433401e+00  1.04795590e-01  1.16116273e+00]
[-2.86187744e+02  1.59144608e+02  2.53865643e+01  1.48403635e+01
  2.00531025e+01  1.59656124e+01  2.46856958e-02  2.30877662e+00
  2.62519002e+00 -1.10081995e+00 -1.00896149e+01  6.85040808e+00
  4.45368528e+00 -4.11487961e+00  3.77111149e+00  1.63347983e+00
  1.22343516e+00 -2.25666666e+00  3.82052279e+00  1.57108784e+00
 -4.55190754e

/tmp/ipykernel_41098/2064313628.py:22: FutureWarning: Pass y=[-0.8614807  -0.8739929  -0.88882446 ...  0.03527832  0.03814697
  0.04098511] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


array(['fearful'], dtype='<U7')